## Web-Content-Q-A-Tool

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]


In [ ]:
from langchain_openai import ChatOpenAI
llmModel = ChatOpenAI(openai_api_key=openai_api_key)

In [15]:
llmModel.model_name

'gpt-3.5-turbo'

In [16]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

page_url = "https://docs.streamlit.io/develop/concepts/architecture/architecture"

loader = WebBaseLoader(web_paths=[page_url], 
                       bs_kwargs=dict(parse_only=bs4.SoupStrainer("main"))) # only elements in main tag
docs = loader.load()
print(docs[0].page_content[:500])

DocumentationsearchSearchrocket_launchGet startedInstallationaddFundamentalsaddFirst stepsaddcodeDevelopConceptsremoveCOREArchitecture & executionremoveRunning your appStreamlit's architectureThe app chromeCachingSession StateFormsFragmentsWidget behaviorMultipage appsaddApp designaddADDITIONALConnections, secrets, & authenticationaddCustom componentsaddConfiguration & themingaddApp testingaddAPI referenceaddTutorialsaddQuick referenceaddweb_assetDeployConceptsaddStreamlit Community CloudaddSnow


In [17]:
print(docs[0].page_content[:500])

DocumentationsearchSearchrocket_launchGet startedInstallationaddFundamentalsaddFirst stepsaddcodeDevelopConceptsremoveCOREArchitecture & executionremoveRunning your appStreamlit's architectureThe app chromeCachingSession StateFormsFragmentsWidget behaviorMultipage appsaddApp designaddADDITIONALConnections, secrets, & authenticationaddCustom componentsaddConfiguration & themingaddApp testingaddAPI referenceaddTutorialsaddQuick referenceaddweb_assetDeployConceptsaddStreamlit Community CloudaddSnow


In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
splits = text_splitter.split_documents(docs)

In [19]:
len(splits)

5

In [20]:
from pprint import pprint

pprint(splits[:5])

[Document(metadata={'source': 'https://docs.streamlit.io/develop/concepts/architecture/architecture', 'start_index': 0}, page_content="DocumentationsearchSearchrocket_launchGet startedInstallationaddFundamentalsaddFirst stepsaddcodeDevelopConceptsremoveCOREArchitecture & executionremoveRunning your appStreamlit's architectureThe app chromeCachingSession StateFormsFragmentsWidget behaviorMultipage appsaddApp designaddADDITIONALConnections, secrets, & authenticationaddCustom componentsaddConfiguration & themingaddApp testingaddAPI referenceaddTutorialsaddQuick referenceaddweb_assetDeployConceptsaddStreamlit Community CloudaddSnowflakeOther platformsaddschoolKnowledge baseFAQInstalling dependenciesDeployment issuesHome/Develop/Concepts/Architecture & execution/Streamlit's architectureUnderstanding Streamlit's client-server architecture"),
 Document(metadata={'source': 'https://docs.streamlit.io/develop/concepts/architecture/architecture', 'start_index': 710}, page_content="Streamlit apps 

In [21]:
len(splits[0].page_content)

709

# Vector Strore

In [22]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY))

# Retrieve

In [23]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [24]:
retrieved_docs = retriever.invoke("how can I trim messages?")

Number of requested results 6 is greater than number of elements in index 5, updating n_results = 5


In [25]:
len(retrieved_docs)

5

In [26]:
print(retrieved_docs[3].page_content)

Browser frontend (client)
When someone views your app through a browser, their device is a Streamlit client. When you view your app from the same computer where you are running or developing your app, then server and client are coincidentally running on the same machine. However, when users view your app across a local network or the internet, the client runs on a different machine from the server.
Server-client impact on app design
Keep in mind the following considerations when building your Streamlit app:


In [27]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Write with simple language. Write at least least 5 sentences.

{context}

Question: {question}

Helpful Answer:"""

prompt = PromptTemplate.from_template(template)

In [28]:

from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0,openai_api_key=OPENAI_API_KEY)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [29]:
from pydantic import BaseModel

class Maintain(BaseModel):
    class Config:
        arbitrary_types_allowed = True

In [30]:
rag_chain.invoke("how do i trim?")

Number of requested results 6 is greater than number of elements in index 5, updating n_results = 5


"I'm sorry, but based on the provided context, it's unclear what you're asking to trim. The context discusses the architecture and operation of Streamlit apps, focusing on the client-server structure, how apps run, and considerations for app development. If you're asking about trimming in the context of programming or app development with Streamlit, could you please provide more specific details? For example, trimming could refer to reducing the size of your app, optimizing code, or manipulating strings or data. Without more specific information, I can't provide a helpful answer."